In [192]:
import pandas as pd
import numpy as np
import requests
import urllib
import re
import string
from bs4 import BeautifulSoup

In [11]:
#Open the page with all albums.
main_page = 'https://www.azlyrics.com/a/atmosphere.html'
try:
    content = urllib.request.urlopen(main_page).read()
    main_soup = BeautifulSoup(content, 'html.parser')
except Exception as e:
    print('Error'+str(e))
#Pull the list of albums on the page and the associated IDs.
album_list = main_soup.find(id='listAlbum')
albums = [name.string.strip('\"') for name in album_list.findAll('b')]
album_ids = [number.get('id') for number in album_list.findAll(['a']) if len(number)==0]

In [104]:
#Create a dataframe containing each album and the ID
albumdf = pd.DataFrame(data={'AlbumName': albums, 'AlbumID': album_ids})
albumdf

,AlbumName,AlbumID
0,Overcast!,7391
1,Lucy Ford: The Atmosphere EPs,5001
2,God Loves Ugly,5005
3,Seven's Travels,3262
4,Headshots: SE7EN,7592
5,You Can't Imagine How Much Fun We're Having,5146
6,Happy Clown Bad Dub 8 / Fun EP,7187
7,Sad Clown Bad Summer #9,6298
8,Sad Clown Bad Fall #10,7048
9,Sad Clown Bad Winter #11,7050


In [164]:
#Need to implement loop/vectorize (if possible) so it can be done on every album.
album_dicts = dict.fromkeys(albumdf['AlbumID'])
for i,j in enumerate(albumdf['AlbumID']):
    #Find the album
    findalbum = album_list.findAll(class_='album')[i]
    #Initialize song list
    songs = []
    #Find where the song titles begin
    title = findalbum.findNext('a', text=True)
    #Loop through the list until reaching the next album, where there is an ID field.
    if i < len(albumdf)-1:
        while title.get('id') != albumdf['AlbumID'][i+1]:
            songs.append(title.text)
            title = title.findNext('a')
        album_dicts[j] = songs
    else:
        while title.text != 'A Long Hello':
            songs.append(title.text)
            title = title.findNext('a')
        album_dicts[j] = songs

In [169]:
#Add song counts to the albumdf
albumdf['Songs'] = albumdf['AlbumID'].apply(lambda idn: len(album_dicts[idn]))

In [215]:
all_songs = [name for sublist in album_dicts.values() for name in sublist]
songdf = pd.DataFrame({'SongName': all_songs})
songdf['SongName']

0                                             1597
1                                Brief Description
2                                   Current Status
3                                    Complications
4                                          4:30 AM
5                                           Adjust
6                                             Clay
7                               Sound Is Vibration
8                                        Multiples
9                                        Scapegoat
10                                             WND
11                             Multiples (Reprise)
12                                        Caved In
13                           Cuando Limpia El Humo
14                                    The Outernet
15                                          Primer
16                               Between The Lines
17                                      Like Today
18                             Tears For The Sheep
19                             

In [213]:
#Function to clean up the song names
def clean_names(string):
    pattern = re.compile('[\W_]+')
    string = pattern.sub('', string.lower())
    return string

In [222]:
#Create a new column with the cleaned up names
songdf['CleanedName'] = songdf['SongName'].apply(lambda song: clean_names(song))
#And another for the different song pages
songdf['SongPage'] = songdf['CleanedName'].apply(
    lambda name: 'https://www.azlyrics.com/lyrics/atmosphere/'+name+'.html')
songdf

,SongName,CleanedName,SongPage
0,1597,1597,https://www.azlyrics.com/lyrics/atmosphere/159...
1,Brief Description,briefdescription,https://www.azlyrics.com/lyrics/atmosphere/bri...
2,Current Status,currentstatus,https://www.azlyrics.com/lyrics/atmosphere/cur...
3,Complications,complications,https://www.azlyrics.com/lyrics/atmosphere/com...
4,4:30 AM,430am,https://www.azlyrics.com/lyrics/atmosphere/430...
5,Adjust,adjust,https://www.azlyrics.com/lyrics/atmosphere/adj...
6,Clay,clay,https://www.azlyrics.com/lyrics/atmosphere/cla...
7,Sound Is Vibration,soundisvibration,https://www.azlyrics.com/lyrics/atmosphere/sou...
8,Multiples,multiples,https://www.azlyrics.com/lyrics/atmosphere/mul...
9,Scapegoat,scapegoat,https://www.azlyrics.com/lyrics/atmosphere/sca...


In [224]:
#Next, get a lexicon for every song by iterating through the pages.
#Also find a way to associate an album with every song in the songdf.

In [236]:
#Find key based on name in a list? -> extend this idea.
k1 = list(album_dicts.keys())[0]
s = '1597'
[s in album_dicts[k1]]

[True]